In [1]:
import xarray as xr
from seapopym_optimization import NoTransportCostFunction, Observation, FunctionalGroupOptimizeNoTransport
import pint

from seapopym.configuration.no_transport.parameter import ForcingParameters

from seapopym.configuration.parameters.parameter_forcing import ForcingUnit

from seapopym.standard.units import StandardUnitsLabels

Load forcing.


In [2]:
time_start, time_end = "1998-01-01", "2022-01-01"

bats_data = xr.open_dataset("../notebooks/1_data_processing/1_1_Forcing/bats_cmems.zarr", engine="zarr")
bats_data["T"].attrs["units"] = StandardUnitsLabels.temperature.units
bats_data = bats_data.sel(time=slice(time_start, time_end))
_ = bats_data.load()

cafe_npp = xr.open_dataset("../notebooks/1_data_processing/1_1_Forcing/bats_cafe.zarr", engine="zarr")
cafe_npp = cafe_npp.sel(time=slice(time_start, time_end))
cafe_npp = cafe_npp.dropna("time")
cafe_npp = cafe_npp.resample(time="D").interpolate("linear")
cafe_npp.lat.attrs = {"units": "degrees_north", "long_name": "latitude", "axis": "Y"}
cafe_npp.lon.attrs = {"units": "degrees_east", "long_name": "longitude", "axis": "X"}
cafe_npp = cafe_npp.rename({"lat": "latitude", "lon": "longitude"})
_ = cafe_npp.load()

In [3]:
bats_data

<xarray.Dataset> Size: 807kB
Dimensions:              (time: 8767, depth: 3, latitude: 1, longitude: 1)
Coordinates:
  * depth                (depth) float32 12B 1.0 2.0 3.0
  * latitude             (latitude) float64 8B 31.6
  * longitude            (longitude) float64 8B -64.2
  * time                 (time) datetime64[ns] 70kB 1998-01-01 ... 2022-01-01
Data variables: (12/13)
    T                    (time, depth, latitude, longitude) float32 105kB 20....
    U                    (time, depth, latitude, longitude) float32 105kB 0.1...
    V                    (time, depth, latitude, longitude) float32 105kB -0....
    mnkc_epi             (time, latitude, longitude) float32 35kB 0.7599 ... ...
    mnkc_hmlmeso         (time, latitude, longitude) float32 35kB 0.6979 ... ...
    mnkc_lmeso           (time, latitude, longitude) float32 35kB 1.093 ... 1...
    ...                   ...
    mnkc_mumeso          (time, latitude, longitude) float32 35kB 0.6176 ... ...
    mnkc_umeso           (time, latitude, longitude) float32 35kB 0.3953 ... ...
    npp                  (time, latitude, longitude) float32 35kB 446.1 ... 2...
    pelagic_layer_depth  (time, depth, latitude, longitude) float32 105kB 100...
    zeu                  (time, latitude, longitude) float32 35kB 66.99 ... 7...
    zooc                 (time, latitude, longitude) float32 35kB 0.3641 ... ...
Attributes:
    Conventions:       CF-1.11
    history:           Created on 2022-09-28
    institution:       CLS
    references:        http://www.cls.fr; http://www.seapodym.eu
    source:            SEAPODYM-LMTL 3.0.0
    subset:datasetId:  cmems_mod_glo_bgc_my_0.083deg-lmtl_PT1D-i_202211
    subset:date:       2024-06-12T03:45:47.386Z
    subset:productId:  GLOBAL_MULTIYEAR_BGC_001_033
    subset:source:     ARCO data downloaded from the Marine Data Store using ...
    title:             Global ocean low and mid trophic levels biomass conten...

Load observations.

First I multiply the observations by the average epipelagic layer depth (150m) to have a biomass in m2 rather than m3.


In [4]:
def update_layer(data: xr.DataArray, epipelagic_size):
    data = data.pint.quantify() * epipelagic_size
    layer_attrs = data["layer"].attrs
    data = data.assign_coords({"layer": [1]})
    data["layer"].attrs = layer_attrs
    return data


epipelagic_size = 150 * pint.application_registry("meter")

obs_bats = xr.open_dataset("../../seapopym-data/data/zooplankton_product/Bats_zooplankton.nc")
obs_bats = update_layer(obs_bats, epipelagic_size)
obs_bats = Observation(obs_bats)

obs_hot = xr.open_dataset("../../seapopym-data/data/zooplankton_product/Hot_zooplankton.nc")
obs_hot = update_layer(obs_hot, epipelagic_size)
obs_hot = Observation(obs_hot)

obs_papa = xr.open_dataset("../../seapopym-data/data/zooplankton_product/Papa_zooplankton.nc")
obs_papa = update_layer(obs_papa, epipelagic_size)
obs_papa = Observation(obs_papa)

Create structure for SeapoPym simulation.


In [5]:
forcing_parameters = ForcingParameters(
    temperature=ForcingUnit(forcing=bats_data["T"], resolution=1 / 12),
    primary_production=ForcingUnit(forcing=cafe_npp["CAFE"], resolution=1 / 12),
)

2024-10-11 00:26:36,417 :: Seapodym ::  WARNING ::
|	CAFE unit is milligram / day / meter ** 2, it will be converted to kilogram / day / meter ** 2.



Setup the cost function.


In [9]:
functional_groups = [
    FunctionalGroupOptimizeNoTransport(day_layer=1, night_layer=1, name="D1N1"),
    FunctionalGroupOptimizeNoTransport(day_layer=2, night_layer=1, name="D2N1"),
]
cost_function = NoTransportCostFunction(
    functional_groups=functional_groups,
    forcing_parameters=forcing_parameters,
    observations=[obs_bats],
).generate()

And execute the process.


In [10]:
cost = cost_function([11, 0.1, 100, -0.1, 0.2, 11, 0.1, 100, -0.1, 0.2])

2024-10-11 00:27:21,341 :: Seapodym ::  WARNING ::
|	Parameter inv_lambda_rate : -0.1 has a negative value. It means that the mortality is decreasing when temperature increase. Do you mean to use a positive value?

2024-10-11 00:27:21,342 :: Seapodym ::  WARNING ::
|	Parameter temperature_recruitment_rate : 0.1 has a positive value. It means that the recruitment time is increasing with temperature. Do you mean to use a negative value?

2024-10-11 00:27:21,343 :: Seapodym ::  WARNING ::
|	Parameter inv_lambda_rate : -0.1 has a negative value. It means that the mortality is decreasing when temperature increase. Do you mean to use a positive value?

2024-10-11 00:27:21,343 :: Seapodym ::  WARNING ::
|	Parameter temperature_recruitment_rate : 0.1 has a positive value. It means that the recruitment time is increasing with temperature. Do you mean to use a negative value?

2024-10-11 00:27:21,393 :: Seapodym ::  DEBUG ::
|	Direct computation for global_mask_from_nan.

2024-10-11 00:27:21,397

Finaly here is the result :


In [11]:
cost

(0.007737407375442469,)